# Analysis of CIA Factbook Data

## Introduction and Import of Data

The data for this analysis comes from the CIA factbook, more information on this data can be found at https://www.cia.gov/the-world-factbook/

This analysis will explore the data in the factbook and take a closer look at population and area information for countries around the world and see if there are any interesting insights to be made.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Taking a quick look at the tables in the database.

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Overview of Data

In [2]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


This gives us a quick look at what to expect in the database, the descriptions for the columns are as follows:

<ul>
<li>code - Abbreviation of the country name</li>
<li>name - Thge name of the country</li>
<li>area - The country's total area (both land and water)</li>
<li>area_land - The country's land area in square kilometers</li>
<li>area_water - The country's water area in square kilometers</li>
<li>population - The country's population</li>
<li>population_growth - The country's population growth as a percentage</li>
<li>birth_rate - The country's number of births per 1,000 people</li>
<li>death_rate - The country's number of deaths per 1,000 people</li>
<li>migration_rate - The number of migrants per 1,000 people (positive = net immigration)</li>
</ul>

## Summary Statistics

In [4]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


Immediatly it sticks out to see that there is a country with a population listed as 0, and that another country is listed as having over 7 billion people. It would be interesting to take a look at what countries are showing these numbers.

## Exploring Outliers

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population)
                   FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the country with 0 population turns out to be Antarctica, which makes sense knowing that there are no permanent residents there and only teams of scientists at research stations, but since none of the scientists live permanently at these stations they would not count for the population. 

In [6]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                   FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The large number seems to come from the 'World' entry, or the total of all of the countries included in the data.

Now that we know what is causing these two outliers we can take another look at the data without including the total world data.

## Recalculating Summary Statistics

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
FROM facts
WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Now that the total for the world is not included we can see that there is a country with a population of almost 1.4 billion!

## Average Population and Area

One thing to look at with this data would be the population density of different countries. The density relies of the population and area of a country, so we will take a look at the averages of those values for the data, excluding the values for the whole world.

In [8]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
FROM facts
WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


The average population is roughly 32 million, and the average area is about 555,000 square kilometers.

## Identifying Densely Populated Countries

Now that we have looked at the average population and area of the countries of the world we can take a look at countries that are densely populated. We'll identify countries that have greater than average values for population, and less than average values for area.

In [9]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World')
AND area < (SELECT AVG(area) FROM facts WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Now that we know which countries have a higher than average population or a smaller than average area, we can compare this to countries that have a high population/area ratio and see how the lists compare.

In [10]:
%%sql
SELECT name, population/area AS pop_density
FROM facts
ORDER BY pop_density DESC
LIMIT 15;

Done.


name,pop_density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


It looks like Bangladesh shows up on both lists, but the rest of the countries with the highest population/area ratio do not have a higher than average population or a lower than average total area.

We'll take another look and see how this list changes when looking only at the countries land area.

In [11]:
%%sql
SELECT name, population/area_land AS pop_density
FROM facts
ORDER BY pop_density DESC
LIMIT 15;

Done.


name,pop_density
Macau,21168
Monaco,15267
Singapore,8259
Hong Kong,6655
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


It looks like the list remains mostly the same, with a few places being swithched and Taiwan entering the list in place of Barbados.

## Exploring Country Area

Now we'll take a look at the top 10 countries with the largest area, followed by the top 10 countries with the lowest area.

In [12]:
%%sql
SELECT name, area
FROM facts
ORDER BY area DESC
LIMIT 10;

Done.


name,area
Russia,17098242
Canada,9984670
United States,9826675
China,9596960
Brazil,8515770
Australia,7741220
European Union,4324782
India,3287263
Argentina,2780400
Kazakhstan,2724900


In [13]:
%%sql
SELECT name, area
FROM facts
ORDER BY area ASC
LIMIT 10;

Done.


name,area
Chad,None
Niger,None
French Southern and Antarctic Lands,None
Saint Barthelemy,None
United States Pacific Island Wildlife Refuges,None
Antarctica,None
Arctic Ocean,None
Atlantic Ocean,None
Indian Ocean,None
Pacific Ocean,None


Interesting to see that there are some countries with no listed area value. We'll take a look at this again but excluding any entries without a value.

In [15]:
%%sql
SELECT name, area
FROM facts
WHERE area NOT Null
ORDER BY area ASC
LIMIT 10;

Done.


name,area
Holy See (Vatican City),0
Monaco,2
Coral Sea Islands,3
Ashmore and Cartier Islands,5
Navassa Island,5
Spratly Islands,5
Clipperton Island,6
Gibraltar,6
Wake Island,6
Paracel Islands,7


This gives us a much better look at the smallest countries by area, made up of islands and small city states.

The next aspect of area we'll look at is seeing which countries have a larger water area than land area.

In [16]:
%%sql
SELECT name, area, area_land, area_water
FROM facts
WHERE area_water > area_land;

Done.


name,area,area_land,area_water
British Indian Ocean Territory,54400,60,54340
Virgin Islands,1910,346,1564


I thought that a few more would show up on this list, but not surprising to see island countries here.

## Exploring Country Population

Next we will take a look at country population information starting by looking at which countries have a higher birth rate than death rate.

In [18]:
%%sql
SELECT *
FROM facts
WHERE birth_rate > death_rate
ORDER BY birth_rate DESC
LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
194,za,Zambia,752618,743398,9220,15066266,2.88,42.13,12.67,0.68
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
160,so,Somalia,637657,627337,10320,10616380,1.83,40.45,13.62,8.49
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
121,mz,Mozambique,799380,786380,13000,25303113,2.45,38.58,12.1,1.98


Interesting to see all of the top 10 being African countries. We'll look at the top 10 countries by population growth next and see how the lists compare.

In [19]:
%%sql
SELECT *
FROM facts
ORDER BY population_growth DESC
LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62


Unsurprisingly many of the countries above show up here, but also with some other countries in the list, most notably Qatar with a migration rate of 22.39! Seems that a lot of people are moving there, let's see what the top 10 migration rates look like to see how much that stands out from the rest.

In [20]:
%%sql
SELECT *
FROM facts
ORDER BY migration_rate DESC
LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
242,aq,American Samoa,199,199,0,54343,0.3,22.89,4.75,21.13
115,fm,"Micronesia, Federated States of",702,702,0,105216,0.46,20.54,4.23,20.93
170,sy,Syria,185180,183630,1550,17064854,0.16,22.17,4.0,19.79
176,tn,Tonga,747,717,30,106501,0.03,23.0,4.85,17.84
229,vi,British Virgin Islands,151,151,0,33454,2.32,10.91,4.99,17.28
103,lu,Luxembourg,2586,2586,0,570252,2.13,11.37,7.24,17.16
230,cj,Cayman Islands,264,264,0,56092,2.1,12.11,5.53,14.4
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05
123,nr,Nauru,21,21,0,9540,0.55,24.95,5.87,13.63


It looks like there are a few other countries with migration rates in the 20's, and some in the high teens but not all are seeing the same population growth.

Now we'll take a look at the opposite of the first part of this section and see what countries have a higher death rate than birth rate, as well as seeing if they are seeing a negative population growth.

In [21]:
%%sql
SELECT *
FROM facts
WHERE death_rate > birth_rate
LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33


In [24]:
%%sql
SELECT *
FROM facts
WHERE population_growth NOT Null
ORDER BY population_growth ASC
LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
203,nf,Norfolk Island,36,36,0,2210,0.01,None,None,None
221,tl,Tokelau,12,12,0,1337,0.01,None,None,None
232,fk,Falkland Islands (Islas Malvinas),12173,12173,0,3361,0.01,10.9,4.9,None
72,gy,Guyana,214969,196849,18120,735222,0.02,15.59,7.32,8.06
157,lo,Slovakia,49035,48105,930,5445027,0.02,9.91,9.74,0.04
